## ** 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [2]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
직접 투표는 각 분류기별 예측을 모아 다수결 투표로 정해지는 반면, 간접 투표는 각 분류기의 예측 확률을 평균내었을 때 가장 높은 확률의 클래스로 정해진다.

___



### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
학습률을 감소시킨다.
___

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [1]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [4]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [30]:
# model fitting
rf_clf = RandomForestClassifier(n_estimators = 100, random_state=42)
ex_clf = ExtraTreesClassifier(n_estimators = 100, random_state=42)
svc_clf = LinearSVC(max_iter=100, random_state=42)
MLP_clf = MLPClassifier(random_state=42)

rf_clf.fit(X_train, y_train)
ex_clf.fit(X_train, y_train)
svc_clf.fit(X_train, y_train)
MLP_clf.fit(X_train, y_train)

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [7]:
from sklearn.ensemble import VotingClassifier

In [31]:
# model fitting
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('ex', ex_clf), ('svc', svc_clf),('mlp', MLP_clf)], 
    voting = 'hard'
)

voting_clf.fit(X_train, y_train)

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ex', ExtraTreesClassifier(random_state=42)),
                             ('svc', LinearSVC(max_iter=100, random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [32]:
# model test
voting_clf.score(X_test, y_test)

0.969

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [38]:
# 각 분류 모델의 성능 확인
print(rf_clf.score(X_train, y_train))
print(ex_clf.score(X_train, y_train))
print(svc_clf.score(X_train, y_train))
print(MLP_clf.score(X_train, y_train))



1.0
1.0
0.8829591836734694
0.9947959183673469


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. svc

In [39]:
# 가장 성능이 낮은 모델 제거

del voting_clf.estimators_[2]

In [40]:
# model fitting
voting_clf.fit(X_test, y_test)

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ex', ExtraTreesClassifier(random_state=42)),
                             ('svc', LinearSVC(max_iter=100, random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [41]:
# 모델 제거 후 성능 확인
voting_clf.score(X_test, y_test)

0.9978095238095238

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [47]:
# model fitting
rf_clf = RandomForestClassifier(n_estimators = 100, random_state=42)
ex_clf = ExtraTreesClassifier(n_estimators = 100, random_state=42)
svc_clf = LinearSVC(random_state=42)
MLP_clf = MLPClassifier(random_state=42)

rf_clf.fit(X_train, y_train)
ex_clf.fit(X_train, y_train)
svc_clf.fit(X_train, y_train)
MLP_clf.fit(X_train, y_train)

rf_pred = rf_clf.predict(X_test)
ex_pred = ex_clf.predict(X_test)
svc_pred = ex_clf.predict(X_test)
MLP_pred = MLP_clf.predict(X_test)

C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\홍예원\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [48]:
estimators = estimators = [rf_clf, ex_clf, svc_clf, MLP_clf]
X_test_p = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_p[:, index] = estimator.predict(X_test)

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [49]:
# 최종 분류기 모델 생성
rf_final = RandomForestClassifier(n_estimators = 100, random_state=42)

# 최종 분류기 학습 및 예측
rf_final.fit(X_test_p, y_test)
final_pred = rf_final.predict(X_test_p)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [50]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성

pred = np.array([rf_pred, ex_pred, svc_pred, MLP_pred])
print(pred.shape)

# transpose를 이용해 행과 열의 위치 교환. 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦. 
pred = np.transpose(pred)
print(pred.shape)

(4, 21000)
(21000, 4)


In [51]:
# 새로운 데이터셋을 이용하여 블렌더로 예측

rf_final.fit(pred, y_test)
final_pred = rf_final.predict(pred)

In [52]:
# model test

from sklearn.metrics import accuracy_score

accuracy_score(final_pred, y_test)

0.9776666666666667